# 🤖 Machine Learning & Predictive Analytics

## 🎯 **Advanced ML Models for Library Intelligence**

Building sophisticated predictive models using our enhanced analytics database to drive actionable business insights.

### 🧠 **ML Model Portfolio**

#### **📈 Demand Forecasting Models**
- **Book Popularity Prediction**: Which books will be most requested next month?
- **Seasonal Demand Patterns**: Inventory optimization for peak periods
- **Branch-Level Forecasting**: Tailored predictions for each library location

#### **👤 Member Behavior Models**
- **Churn Prediction**: Identify members at risk of leaving (90%+ accuracy target)
- **Late Return Prediction**: Proactive intervention for overdue items
- **Engagement Scoring**: Member lifetime value and engagement potential

#### **🎯 Recommendation Systems**
- **Collaborative Filtering**: "Members like you also enjoyed..."
- **Content-Based Filtering**: Genre and author similarity matching
- **Hybrid Recommendations**: Combined approach for 25%+ improvement

#### **📊 Operational Models**
- **Staff Scheduling Optimization**: Predict busy periods for optimal staffing
- **Collection Optimization**: Which books to acquire/retire based on data
- **Revenue Prediction**: Membership and penalty revenue forecasting

### 🎖️ **Expected Business Impact**
- **25%+ improvement** in recommendation relevance
- **40%+ better inventory** management and demand prediction
- **30%+ reduction** in member churn through early intervention
- **20%+ efficiency gains** in operational resource allocation

---
*Leveraging our world-class enhanced analytics database for predictive excellence*

In [ ]:
# ML Environment Setup - Simplified
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🤖 **MACHINE LEARNING ENVIRONMENT READY**")
print("=" * 50)
print("🎯 Core libraries loaded successfully")
print("📊 Visualization libraries configured")

# Connect to our enhanced database
try:
    conn = sqlite3.connect('library.db')
    print("🔌 Connected to enhanced analytics database")
    
    # Verify enhanced tables exist
    tables_query = "SELECT name FROM sqlite_master WHERE type='table'"
    tables = pd.read_sql_query(tables_query, conn)
    
    enhanced_tables = ['Publisher', 'Member_Preferences', 'Item_Reservations', 'Item_Reviews', 'Daily_Operations_Summary']
    available_enhanced = [t for t in enhanced_tables if t in tables['name'].values]
    
    print(f"✅ Enhanced analytics tables available: {len(available_enhanced)}/5")
    print(f"🚀 Ready for ML modeling with enhanced data!")
    
except Exception as e:
    print(f"❌ Database connection error: {e}")
    print("? Please ensure library.db exists in the current directory")

## 📊 **Phase 1: Data Preparation & Feature Engineering**

### Building the foundation for world-class predictive models

In [ ]:
# Enhanced Data Loading & Feature Engineering
print("📊 **LOADING ENHANCED DATASET FOR ML**")
print("=" * 50)

# Load core data with advanced joins
ml_dataset_query = """
SELECT 
    -- Member Information
    m.Member_ID,
    m.Member_Type,
    m.Join_Date,
    julianday('now') - julianday(m.Join_Date) as Days_Since_Joining,
    
    -- Member Preferences (from enhanced table)
    COALESCE(mp.Reading_Level, 'Unknown') as Reading_Level,
    COALESCE(mp.Preferred_Format, 'Physical') as Preferred_Format,
    COALESCE(mp.Email_Notifications, 0) as Email_Opt_In,
    COALESCE(mp.Privacy_Level, 'Standard') as Privacy_Level,
    
    -- Loan History Aggregations
    COUNT(DISTINCT l.Loan_ID) as Total_Loans,
    COUNT(DISTINCT l.Item_ID) as Unique_Items_Borrowed,
    AVG(julianday(l.Return_Date) - julianday(l.Loan_Date)) as Avg_Loan_Duration,
    SUM(CASE WHEN l.Return_Date > l.Due_Date THEN 1 ELSE 0 END) as Late_Returns,
    SUM(CASE WHEN l.Return_Date IS NULL AND l.Due_Date < date('now') THEN 1 ELSE 0 END) as Current_Overdue,
    
    -- Financial Metrics
    COALESCE(SUM(l.Penalty_Amount), 0) as Total_Penalties,
    
    -- Recent Activity (last 90 days)
    COUNT(CASE WHEN l.Loan_Date >= date('now', '-90 days') THEN 1 END) as Recent_Loans,
    MAX(l.Loan_Date) as Last_Loan_Date,
    julianday('now') - julianday(MAX(l.Loan_Date)) as Days_Since_Last_Loan,
    
    -- Engagement Metrics
    COUNT(DISTINCT ir.Review_ID) as Total_Reviews,
    AVG(CAST(ir.Rating as FLOAT)) as Avg_Rating_Given,
    COUNT(DISTINCT res.Reservation_ID) as Total_Reservations
    
FROM Member m
LEFT JOIN Member_Preferences mp ON m.Member_ID = mp.Member_ID
LEFT JOIN Loan l ON m.Member_ID = l.Member_ID
LEFT JOIN Item_Reviews ir ON m.Member_ID = ir.Member_ID
LEFT JOIN Item_Reservations res ON m.Member_ID = res.Member_ID
GROUP BY m.Member_ID, m.Member_Type, m.Join_Date, mp.Reading_Level, 
         mp.Preferred_Format, mp.Email_Opt_In, mp.Privacy_Level
"""

# Load the enhanced dataset
ml_data = pd.read_sql_query(ml_dataset_query, conn)

print(f"✅ Loaded {len(ml_data)} member records with enhanced features")
print(f"📊 Dataset shape: {ml_data.shape}")
print(f"🔍 Features available: {ml_data.columns.tolist()}")

# Handle missing values intelligently
ml_data['Days_Since_Last_Loan'] = ml_data['Days_Since_Last_Loan'].fillna(9999)  # Never borrowed
ml_data['Avg_Loan_Duration'] = ml_data['Avg_Loan_Duration'].fillna(0)
ml_data['Avg_Rating_Given'] = ml_data['Avg_Rating_Given'].fillna(0)

# Create advanced engineered features
print("\n🔧 **ADVANCED FEATURE ENGINEERING**")

# Engagement Score (composite metric)
ml_data['Engagement_Score'] = (
    ml_data['Total_Loans'] * 0.3 +
    ml_data['Total_Reviews'] * 0.2 +
    ml_data['Total_Reservations'] * 0.1 +
    (ml_data['Email_Opt_In'] * 10) * 0.1 +
    (10 - np.minimum(ml_data['Days_Since_Last_Loan'] / 10, 10)) * 0.3
)

# Risk Indicators
ml_data['Late_Return_Rate'] = ml_data['Late_Returns'] / np.maximum(ml_data['Total_Loans'], 1)
ml_data['High_Risk'] = (
    (ml_data['Late_Return_Rate'] > 0.3) | 
    (ml_data['Days_Since_Last_Loan'] > 180) |
    (ml_data['Total_Penalties'] > 50)
).astype(int)

# Member Lifecycle Stage
def member_lifecycle(row):
    if row['Days_Since_Joining'] < 30:
        return 'New'
    elif row['Days_Since_Last_Loan'] > 365:
        return 'Dormant'
    elif row['Days_Since_Last_Loan'] > 90:
        return 'At_Risk'
    elif row['Total_Loans'] > 20 and row['Days_Since_Last_Loan'] < 30:
        return 'Power_User'
    else:
        return 'Active'

ml_data['Lifecycle_Stage'] = ml_data.apply(member_lifecycle, axis=1)

# Value Segmentation
loan_quartiles = ml_data['Total_Loans'].quantile([0.25, 0.5, 0.75])
def value_segment(loans):
    if loans <= loan_quartiles[0.25]:
        return 'Low_Value'
    elif loans <= loan_quartiles[0.5]:
        return 'Medium_Value'
    elif loans <= loan_quartiles[0.75]:
        return 'High_Value'
    else:
        return 'Premium_Value'

ml_data['Value_Segment'] = ml_data['Total_Loans'].apply(value_segment)

print(f"✅ Advanced features engineered:")
print(f"   📊 Engagement Score (0-100 scale)")
print(f"   ⚠️  High Risk Indicator (binary)")
print(f"   🔄 Member Lifecycle Stage (5 categories)")
print(f"   💎 Value Segmentation (4 tiers)")

# Display feature engineering results
print(f"\n📈 **FEATURE ENGINEERING SUMMARY**:")
print(f"   Lifecycle Stages: {ml_data['Lifecycle_Stage'].value_counts().to_dict()}")
print(f"   Value Segments: {ml_data['Value_Segment'].value_counts().to_dict()}")
print(f"   High Risk Members: {ml_data['High_Risk'].sum()}/{len(ml_data)} ({ml_data['High_Risk'].mean():.1%})")
print(f"   Avg Engagement Score: {ml_data['Engagement_Score'].mean():.1f}")

# Preview the enhanced dataset
print(f"\n📋 **ENHANCED ML DATASET PREVIEW**:")
display_cols = ['Member_ID', 'Member_Type', 'Total_Loans', 'Engagement_Score', 
                'Lifecycle_Stage', 'Value_Segment', 'High_Risk']
print(ml_data[display_cols].head(10).to_string(index=False))

## 🎯 **Phase 2: Member Churn Prediction Model**

### Building a high-accuracy model to identify members at risk of leaving

In [ ]:
# Member Churn Prediction Model
print("🎯 **MEMBER CHURN PREDICTION MODEL**")
print("=" * 50)

# Define churn based on multiple criteria
def define_churn(row):
    # Churn if no activity in 6+ months OR very high penalty amounts
    return (
        (row['Days_Since_Last_Loan'] > 180) |
        (row['Total_Penalties'] > 100) |
        (row['Late_Return_Rate'] > 0.5 and row['Total_Loans'] > 5)
    )

ml_data['Churn_Risk'] = ml_data.apply(define_churn, axis=1).astype(int)

print(f"📊 Churn Distribution:")
churn_dist = ml_data['Churn_Risk'].value_counts()
print(f"   Active Members: {churn_dist.get(0, 0)} ({churn_dist.get(0, 0)/len(ml_data):.1%})")
print(f"   At-Risk Members: {churn_dist.get(1, 0)} ({churn_dist.get(1, 0)/len(ml_data):.1%})")

# Prepare features for churn prediction
churn_features = [
    'Days_Since_Joining', 'Total_Loans', 'Unique_Items_Borrowed',
    'Avg_Loan_Duration', 'Late_Return_Rate', 'Total_Penalties',
    'Recent_Loans', 'Days_Since_Last_Loan', 'Total_Reviews',
    'Total_Reservations', 'Engagement_Score', 'Email_Opt_In'
]

# Handle categorical variables
le_member_type = LabelEncoder()
le_reading_level = LabelEncoder()
le_format = LabelEncoder()
le_privacy = LabelEncoder()

ml_data['Member_Type_Encoded'] = le_member_type.fit_transform(ml_data['Member_Type'])
ml_data['Reading_Level_Encoded'] = le_reading_level.fit_transform(ml_data['Reading_Level'])
ml_data['Format_Encoded'] = le_format.fit_transform(ml_data['Preferred_Format'])
ml_data['Privacy_Encoded'] = le_privacy.fit_transform(ml_data['Privacy_Level'])

churn_features.extend(['Member_Type_Encoded', 'Reading_Level_Encoded', 'Format_Encoded', 'Privacy_Encoded'])

# Prepare training data
X_churn = ml_data[churn_features]
y_churn = ml_data['Churn_Risk']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_churn, y_churn, test_size=0.2, random_state=42, stratify=y_churn
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n🎯 **CHURN PREDICTION TRAINING**:")
print(f"   Training samples: {len(X_train)}")
print(f"   Test samples: {len(X_test)}")
print(f"   Features: {len(churn_features)}")

# Train multiple models
churn_models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'SVM': SVC(probability=True, random_state=42)
}

churn_results = {}

for name, model in churn_models.items():
    # Train model
    if name == 'SVM':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    churn_results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'predictions': y_pred,
        'probabilities': y_proba
    }
    
    print(f"\n📊 {name} Results:")
    print(f"   Accuracy: {accuracy:.3f}")
    print(f"   Precision: {precision:.3f}")
    print(f"   Recall: {recall:.3f}")
    print(f"   F1-Score: {f1:.3f}")

# Select best model
best_churn_model = max(churn_results.items(), key=lambda x: x[1]['f1'])
print(f"\n🏆 **BEST CHURN MODEL**: {best_churn_model[0]} (F1: {best_churn_model[1]['f1']:.3f})")

# Feature importance for best model
if best_churn_model[0] in ['Random Forest', 'Gradient Boosting']:
    feature_importance = pd.DataFrame({
        'feature': churn_features,
        'importance': best_churn_model[1]['model'].feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n🔍 **TOP 10 CHURN PREDICTION FEATURES**:")
    for idx, row in feature_importance.head(10).iterrows():
        print(f"   {row['feature']}: {row['importance']:.3f}")

In [ ]:
# Churn Prediction Visualization
print("📊 **CHURN PREDICTION VISUALIZATION**")
print("=" * 50)

# Create visualization dashboard
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Model Performance Comparison
ax1 = axes[0, 0]
model_names = list(churn_results.keys())
f1_scores = [results['f1'] for results in churn_results.values()]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

bars = ax1.bar(model_names, f1_scores, color=colors)
ax1.set_title('🏆 Churn Prediction Model Comparison\n(F1-Score)', fontsize=14, fontweight='bold')
ax1.set_ylabel('F1-Score')
ax1.set_ylim(0, 1)

# Add value labels on bars
for bar, score in zip(bars, f1_scores):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{score:.3f}', ha='center', va='bottom', fontweight='bold')

plt.xticks(rotation=45)

# 2. Feature Importance (if available)
ax2 = axes[0, 1]
if best_churn_model[0] in ['Random Forest', 'Gradient Boosting']:
    top_features = feature_importance.head(8)
    bars = ax2.barh(top_features['feature'], top_features['importance'], color='#FF6B6B')
    ax2.set_title('🔍 Top Churn Prediction Features', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Feature Importance')
else:
    ax2.text(0.5, 0.5, 'Feature Importance\nNot Available\nfor Selected Model',
             ha='center', va='center', transform=ax2.transAxes, fontsize=12)
    ax2.set_title('🔍 Feature Importance', fontsize=14, fontweight='bold')

# 3. Churn Risk Distribution
ax3 = axes[1, 0]
churn_counts = ml_data['Churn_Risk'].value_counts()
labels = ['Active Members', 'At-Risk Members']
sizes = [churn_counts.get(0, 0), churn_counts.get(1, 0)]
colors_pie = ['#4ECDC4', '#FF6B6B']
explode = (0, 0.1)  # explode the at-risk slice

ax3.pie(sizes, labels=labels, colors=colors_pie, autopct='%1.1f%%',
        startangle=90, explode=explode)
ax3.set_title('⚠️ Member Churn Risk Distribution', fontsize=14, fontweight='bold')

# 4. Engagement Score vs Churn Risk
ax4 = axes[1, 1]
churn_0 = ml_data[ml_data['Churn_Risk'] == 0]['Engagement_Score']
churn_1 = ml_data[ml_data['Churn_Risk'] == 1]['Engagement_Score']

ax4.hist([churn_0, churn_1], bins=20, alpha=0.7, 
         label=['Active', 'At-Risk'], color=['#4ECDC4', '#FF6B6B'])
ax4.set_title('📊 Engagement Score Distribution\nby Churn Risk', fontsize=14, fontweight='bold')
ax4.set_xlabel('Engagement Score')
ax4.set_ylabel('Number of Members')
ax4.legend()

plt.tight_layout()
plt.show()

# Actionable Insights
print(f"\n🎯 **ACTIONABLE CHURN INSIGHTS**:")
at_risk_members = ml_data[ml_data['Churn_Risk'] == 1]
print(f"   📈 {len(at_risk_members)} members identified as at-risk")
print(f"   💡 Average days since last loan for at-risk: {at_risk_members['Days_Since_Last_Loan'].mean():.0f}")
print(f"   📧 Email opt-in rate for at-risk: {at_risk_members['Email_Opt_In'].mean():.1%}")
print(f"   💰 Average penalties for at-risk: ${at_risk_members['Total_Penalties'].mean():.2f}")

# Identify top at-risk members for intervention
best_model_proba = best_churn_model[1]['probabilities']
X_test_with_proba = X_test.copy()
X_test_with_proba['Churn_Probability'] = best_model_proba
top_at_risk = X_test_with_proba.nlargest(10, 'Churn_Probability')

print(f"\n🚨 **TOP 10 MEMBERS FOR IMMEDIATE INTERVENTION**:")
print(f"   (Based on highest churn probability)")
for idx, (_, row) in enumerate(top_at_risk.iterrows(), 1):
    member_id = ml_data.iloc[row.name]['Member_ID']
    prob = row['Churn_Probability']
    print(f"   {idx}. Member {member_id}: {prob:.1%} churn risk")

## 📚 **Phase 3: Book Demand Forecasting Model**

### Predicting which books will be most popular for optimal inventory management

In [ ]:
# Book Demand Forecasting Model
print("📚 **BOOK DEMAND FORECASTING MODEL**")
print("=" * 50)

# Load book popularity data
book_demand_query = """
SELECT 
    i.Item_ID,
    i.Title,
    i.Genre,
    i.Publication_Year,
    2025 - i.Publication_Year as Book_Age,
    
    -- Loan metrics
    COUNT(DISTINCT l.Loan_ID) as Total_Loans,
    COUNT(DISTINCT l.Member_ID) as Unique_Borrowers,
    AVG(julianday(l.Return_Date) - julianday(l.Loan_Date)) as Avg_Loan_Duration,
    
    -- Recent popularity (last 90 days)
    COUNT(CASE WHEN l.Loan_Date >= date('now', '-90 days') THEN 1 END) as Recent_Loans,
    
    -- Review metrics
    COUNT(DISTINCT ir.Review_ID) as Total_Reviews,
    AVG(CAST(ir.Rating as FLOAT)) as Avg_Rating,
    COUNT(CASE WHEN ir.Would_Recommend = 1 THEN 1 END) as Recommendations,
    
    -- Reservation metrics
    COUNT(DISTINCT res.Reservation_ID) as Total_Reservations,
    COUNT(CASE WHEN res.Status = 'Active' THEN 1 END) as Active_Reservations
    
FROM Item i
LEFT JOIN Loan l ON i.Item_ID = l.Item_ID
LEFT JOIN Item_Reviews ir ON i.Item_ID = ir.Item_ID
LEFT JOIN Item_Reservations res ON i.Item_ID = res.Item_ID
GROUP BY i.Item_ID, i.Title, i.Genre, i.Publication_Year
HAVING COUNT(DISTINCT l.Loan_ID) > 0  -- Only books that have been borrowed
"""

book_data = pd.read_sql_query(book_demand_query, conn)

print(f"✅ Loaded {len(book_data)} books with demand history")

# Handle missing values
book_data['Avg_Rating'] = book_data['Avg_Rating'].fillna(book_data['Avg_Rating'].mean())
book_data['Avg_Loan_Duration'] = book_data['Avg_Loan_Duration'].fillna(book_data['Avg_Loan_Duration'].mean())

# Create demand categories (target variable)
# High demand: top 25%, Medium: middle 50%, Low: bottom 25%
demand_thresholds = book_data['Total_Loans'].quantile([0.25, 0.75])

def demand_category(loans):
    if loans >= demand_thresholds[0.75]:
        return 'High'
    elif loans >= demand_thresholds[0.25]:
        return 'Medium'
    else:
        return 'Low'

book_data['Demand_Category'] = book_data['Total_Loans'].apply(demand_category)

print(f"\n📊 Demand Distribution:")
demand_dist = book_data['Demand_Category'].value_counts()
for category, count in demand_dist.items():
    print(f"   {category} Demand: {count} books ({count/len(book_data):.1%})")

# Engineer features for demand prediction
# Genre popularity impact
genre_popularity = book_data.groupby('Genre')['Total_Loans'].mean().to_dict()
book_data['Genre_Popularity'] = book_data['Genre'].map(genre_popularity)

# Recency factor (how recent the book is)
book_data['Recency_Score'] = np.maximum(0, 10 - (book_data['Book_Age'] / 5))  # Newer books score higher

# Engagement ratio
book_data['Review_Engagement'] = book_data['Total_Reviews'] / np.maximum(book_data['Total_Loans'], 1)
book_data['Reservation_Ratio'] = book_data['Total_Reservations'] / np.maximum(book_data['Total_Loans'], 1)

# Momentum indicator (recent vs historical performance)
book_data['Momentum'] = book_data['Recent_Loans'] / np.maximum(book_data['Total_Loans'] / 4, 1)  # Quarterly comparison

# Prepare features for demand prediction
demand_features = [
    'Book_Age', 'Unique_Borrowers', 'Avg_Loan_Duration', 'Avg_Rating',
    'Total_Reviews', 'Recommendations', 'Genre_Popularity', 'Recency_Score',
    'Review_Engagement', 'Reservation_Ratio', 'Momentum', 'Active_Reservations'
]

# Encode genre
le_genre = LabelEncoder()
book_data['Genre_Encoded'] = le_genre.fit_transform(book_data['Genre'])
demand_features.append('Genre_Encoded')

# Prepare training data
X_demand = book_data[demand_features]
y_demand = book_data['Demand_Category']

# Encode target variable
le_demand = LabelEncoder()
y_demand_encoded = le_demand.fit_transform(y_demand)

# Split data
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_demand, y_demand_encoded, test_size=0.2, random_state=42, stratify=y_demand_encoded
)

print(f"\n📚 **DEMAND FORECASTING TRAINING**:")
print(f"   Training samples: {len(X_train_d)}")
print(f"   Test samples: {len(X_test_d)}")
print(f"   Features: {len(demand_features)}")

# Train demand forecasting models
demand_models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

demand_results = {}

for name, model in demand_models.items():
    model.fit(X_train_d, y_train_d)
    y_pred_d = model.predict(X_test_d)
    
    accuracy = accuracy_score(y_test_d, y_pred_d)
    f1 = f1_score(y_test_d, y_pred_d, average='weighted')
    
    demand_results[name] = {
        'model': model,
        'accuracy': accuracy,
        'f1': f1,
        'predictions': y_pred_d
    }
    
    print(f"\n📊 {name} Demand Prediction:")
    print(f"   Accuracy: {accuracy:.3f}")
    print(f"   F1-Score: {f1:.3f}")

# Select best demand model
best_demand_model = max(demand_results.items(), key=lambda x: x[1]['f1'])
print(f"\n🏆 **BEST DEMAND MODEL**: {best_demand_model[0]} (F1: {best_demand_model[1]['f1']:.3f})")

# Feature importance for demand prediction
if best_demand_model[0] in ['Random Forest', 'Gradient Boosting']:
    demand_importance = pd.DataFrame({
        'feature': demand_features,
        'importance': best_demand_model[1]['model'].feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n🔍 **TOP DEMAND PREDICTION FEATURES**:")
    for idx, row in demand_importance.head(8).iterrows():
        print(f"   {row['feature']}: {row['importance']:.3f}")

# Predict future high-demand books
all_predictions = best_demand_model[1]['model'].predict(X_demand)
all_probabilities = best_demand_model[1]['model'].predict_proba(X_demand)

# Get high-demand probability (assuming class 1 is High)
high_demand_class_idx = list(le_demand.classes_).index('High')
book_data['High_Demand_Probability'] = all_probabilities[:, high_demand_class_idx]
book_data['Predicted_Demand'] = le_demand.inverse_transform(all_predictions)

# Identify books to prioritize for acquisition/promotion
high_potential_books = book_data[
    (book_data['High_Demand_Probability'] > 0.7) & 
    (book_data['Recent_Loans'] < 5)  # Currently underperforming
].sort_values('High_Demand_Probability', ascending=False)

print(f"\n🎯 **HIGH-POTENTIAL BOOKS FOR PROMOTION**:")
for idx, (_, book) in enumerate(high_potential_books.head(10).iterrows(), 1):
    print(f"   {idx}. {book['Title'][:40]}... | {book['Genre']} | {book['High_Demand_Probability']:.1%} potential")

print(f"\n📈 **DEMAND FORECASTING INSIGHTS**:")
print(f"   🎯 {len(high_potential_books)} books identified as high-potential")
print(f"   📚 Average age of high-potential books: {high_potential_books['Book_Age'].mean():.1f} years")
print(f"   ⭐ Average rating of high-potential books: {high_potential_books['Avg_Rating'].mean():.1f}/5.0")